In [13]:
# # !pip install pynput
import HandTrackingModule as htm
import cv2 
import time 
import numpy as np 
import math 

from pynput.keyboard import Key , Controller
keyboard = Controller()

import HandTrackingModule as htm
print(dir(htm))

# !pip install  pycaw

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'cv2', 'handDetector', 'main', 'mp', 'time']


In [29]:
import cv2
import math
import numpy as np
import HandTrackingModule as htm
from pynput.keyboard import Controller, Key  # Added for volume control

# Initialize constants
wCam, hCam = 1280, 720
minHand = 50  # Minimum hand distance
maxHand = 300  # Maximum hand distance (adjust as needed)
minAngle = -95
maxAngle = 0

# Initialize video capture and set dimensions
cap = cv2.VideoCapture(0)
cap.set(3, wCam)  # Set width
cap.set(4, hCam)  # Set height

# Initialize hand detector
detector = htm.handDetector(detectionCon=0.7)

# Initialize keyboard controller for volume control
keyboard = Controller()

# Initialize variables to keep track of the previous hand length and angle
last_length = None
smoothing_factor = 5  # Adjust to smooth out sudden movements

# Function to draw the hand and bar
def draw_hand_and_bar(img, lmList, angleBarHeight, angleDeg):
    x1, y1 = lmList[4][1], lmList[4][2]
    x2, y2 = lmList[8][1], lmList[8][2]
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

    cv2.circle(img, (x1, y1), 15, (0, 0, 255), cv2.FILLED)
    cv2.circle(img, (x2, y2), 15, (0, 0, 255), cv2.FILLED)
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
    cv2.circle(img, (cx, cy), 15, (0, 0, 255), cv2.FILLED)

    cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)  # Static reference bar
    cv2.rectangle(img, (50, int(angleBarHeight)), (85, 400), (0, 0, 255), cv2.FILLED)  # Dynamic bar

    cv2.putText(img, f'{int(angleDeg)} %', (40, 120), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 3)

# Function to handle volume control based on hand distance
def handle_volume_control(length, last_length):
    if last_length:
        if length > last_length + smoothing_factor:  # Increasing
            keyboard.press(Key.media_volume_up)
            keyboard.release(Key.media_volume_up)
            return 'Volume Up'
        elif length < last_length - smoothing_factor:  # Decreasing
            keyboard.press(Key.media_volume_down)
            keyboard.release(Key.media_volume_down)
            return 'Volume Down'
    return None

# Main loop
while True:
    success, img = cap.read()
    if not success:
        print("Error: Could not read frame.")
        break

    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)

    if len(lmList) != 0:
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        length = math.hypot(x2 - x1, y2 - y1)

        # Interpolate values for angle and bar height
        angleBarHeight = np.interp(length, [minHand, maxHand], [150, 400])  # Height of the bar
        angleDeg = np.interp(length, [minHand, maxHand], [0, 100])

        # Handle volume control and feedback
        feedback = handle_volume_control(length, last_length)
        if feedback:
            cv2.putText(img, feedback, (500, 100), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 3)

        last_length = length

        # Draw hand landmarks and bar
        draw_hand_and_bar(img, lmList, angleBarHeight, angleDeg)

    cv2.imshow('Hand Tracking', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
from ctypes import cast , POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities , IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_,CLSCTX_ALL,None
)

volume = cast(interface, POINTER(IAudioEndpointVolume))
volume.GetVolumeRange()

(-63.5, 0.0, 0.5)

In [27]:
dir(Key)

['__class__',
 '__contains__',
 '__doc__',
 '__getitem__',
 '__init_subclass__',
 '__iter__',
 '__len__',
 '__members__',
 '__module__',
 '__name__',
 '__qualname__',
 'alt',
 'alt_gr',
 'alt_l',
 'alt_r',
 'backspace',
 'caps_lock',
 'cmd',
 'cmd_r',
 'ctrl',
 'ctrl_l',
 'ctrl_r',
 'delete',
 'down',
 'end',
 'enter',
 'esc',
 'f1',
 'f10',
 'f11',
 'f12',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f2',
 'f20',
 'f21',
 'f22',
 'f23',
 'f24',
 'f3',
 'f4',
 'f5',
 'f6',
 'f7',
 'f8',
 'f9',
 'home',
 'insert',
 'left',
 'media_next',
 'media_play_pause',
 'media_previous',
 'media_volume_down',
 'media_volume_mute',
 'media_volume_up',
 'menu',
 'num_lock',
 'page_down',
 'page_up',
 'pause',
 'print_screen',
 'right',
 'scroll_lock',
 'shift',
 'shift_r',
 'space',
 'tab',
 'up']